# Transformer - 기본 과제
> 주어진 문장 다음 단어를 예측하는 모델을 만들어본다
   


### RNN vs Transformer
| 항목             | RNN (Recurrent Neural Network)                             | Transformer                                               |
|:------------------|-------------------------------------------------------------|------------------------------------------------------------|
| **기본 구조**     | 순차적으로 시퀀스를 처리 (시간 순서대로)                  | 전체 시퀀스를 동시에 처리 (병렬처리 가능)                  |
| **병렬 처리**     | ❌ 불가능 (이전 단계 결과가 다음 단계 입력에 필요)         | ✅ 가능 (Self-Attention으로 모든 위치를 동시에 참조)       |
| **장기 의존성 처리** | ❌ 어려움 (Vanishing Gradient 문제)                      | ✅ 훨씬 강력함 (모든 단어 간 관계 파악 가능)               |
| **입력 위치 정보** | 자연스럽게 순서를 따름                                    | 별도로 Positional Encoding 필요                           |
| **대표 모델**     | LSTM, GRU                                                  | BERT, GPT, T5, etc.                                       |
| **성능 및 속도**  | 긴 문장 처리에 약함 / 느림                                 | 긴 문장도 잘 처리 / 빠름 (GPU 병렬화)                      |


   
### IMDb 감성 분류 데이터셋
- Internet Movie Database
- 감성 분류(Sentiment Classification)
  - 영화에 대한 정보 / 사용자 리뷰(text) / 별점
  - 입력: 사용자 리뷰(text)
  - 출력: 감성 라벨(긍정 or 부정)
- 텍스트 분류(Binary Classification) 문제

**Example**
``` makefile
입력: "This movie was surprisingly good and well-acted."
출력: 1 (긍정)
```

``` makefile
입력: "I wasted two hours of my life watching this."
출력: 0 (부정)
```

### import 목록
- `datasets`: Hugging Face의 데이터셋 로더 라이브러리 / IMDb, MNIST 등 다양한 데이터셋 활용 가능
- `sacremoses`: 텍스트 전처리 모듈 / tokenizer 내부에서 사용할 수 있음


In [1]:
!pip install datasets sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux"

# **Attention is all you need**   
![Attention is all you need](https://binarymindset.com/wp-content/uploads/2023/08/transformers-4-574x1024.png)   
- 6 encoder - 6 decoder structure

## Transformer 구조

### 1️⃣Embedding   
### 2️⃣Positional Encoding   
### 3️⃣Encoder   
- Self-Attention
- Feed Forward

### 4️⃣Decoder  
- Masked Self-Attention
- Encoder-Decoder Attention
- Feed Forward

### 5️⃣Prediction   



### Load Tokenizer from Hugging Face

**BERT vs GPT2 Tokenizer | AutoTokenizer** 정리  
[![Notebook](https://img.shields.io/badge/Jupyter-Notebook-orange?logo=jupyter)](https://nbviewer.org/github/zerovodka/ML-learning/blob/master/src/week2/BERT-vs-GPT-Tokenizer.ipynb?flush_cache=true)
- Emoji 처리 관점에서 비교
- AutoTokenizer
- 실무 vs 실험 적절한 import
- tokenizer 내부 성분 값 설명


In [3]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

**Load BERT Tokenizer**   
- BERT Tokenizer Model의 사전 학습된 구조, 설정, vocabulary를 확인할 수 있다

In [21]:
# ds = load_dataset("stanfordnlp/imdb")

# imdb 데이터 앞에서 5%만 잘라서 수행: 빠른 실험을 위함
train_ds = load_dataset("stanfordnlp/imdb", split="train[:5%]")
test_ds = load_dataset("stanfordnlp/imdb", split="test[:5%]")

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

print(tokenizer)

print(f'사용중인 tokenizer 모델: \n{tokenizer.name_or_path}')
print(f'\n\n{tokenizer.name_or_path} 모델의 vacabulary size: \n{tokenizer.vocab_size}')
print(f'\n\n{tokenizer.name_or_path} 모델의 max length: \n{tokenizer.model_max_length}')
print(f'\n\n{tokenizer.name_or_path} 모델의 special_tokens_map: \n{tokenizer.special_tokens_map}')
print(f'\n\n{tokenizer.name_or_path} 모델의 decode 시 특수 token 처리 방식: \n{tokenizer.added_tokens_decoder}')

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
사용중인 tokenizer 모델: 
bert-base-uncased


bert-base-uncased 모델의 vacabulary size: 
30522


bert-base-uncased 모델의 max length: 
512

**Encode function**
- tokenizer 모델 마다 설정 값이 다르기때문에, 각각에 맞는 encode 함수를 선언한다

**기능**
- tokenizing: `text` -> token 쪼개기
- encoding: token -> number ID 인코딩
- padding / truncation: 패딩, 길이 잘라내기(max length 선언 기준)
- return: 딕셔너리 리턴
  - `return_tensor='pt'` 속성 추가 시: python tensor 리턴   

|옵션|설명|
|:---|:---|
|**`return_tensors='pt'`**|	PyTorch 텐서로 리턴 → 모델 입력에 바로 사용 가능|
|**`return_tensors='tf'`**|	TensorFlow용|
|**`return_tensors='np'`**|	NumPy용
|**`return_tensors=None`**|	파이썬 dict (list 형태) → 디버깅/출력 확인에 유리

In [31]:
# is_tensor 값을 넘기지 않으면, LongTensor 처리를 외부에서 따로 해줘야한다
def encode_bert(text, max_len, is_tensor=False):
  return tokenizer(text, padding=True, truncation=True, max_length=max_len, return_tensors= 'pt' if is_tensor else None)

text = 'This movie was incredibly touching and had great performances!! I would definitely recommend it to anyone 🤔🤔.'

print(f'딕셔너리 형태:\n {encode_bert(text, 30)}')
print(f'PyTorch Tensor 형태:\n {encode_bert(text, 30, True)}')

# Tokenizer의 return 타입에 따라 input_ids, token_type_ids, attention_mask의 type이 다르다
# input_ids: input 값이 토큰화 된 것
# attention_mask: 실제 단어: 1 / padding: 0

딕셔너리 형태:
 {'input_ids': [101, 2023, 3185, 2001, 11757, 7244, 1998, 2018, 2307, 4616, 999, 999, 1045, 2052, 5791, 16755, 2009, 2000, 3087, 100, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
PyTorch Tensor 형태:
 {'input_ids': tensor([[  101,  2023,  3185,  2001, 11757,  7244,  1998,  2018,  2307,  4616,
           999,   999,  1045,  2052,  5791, 16755,  2009,  2000,  3087,   100,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


### 배치 데이터 정리 함수
- DataLoader가 train_ds에서 mini_batch 크기 만큼 샘플을 뽑는다
- collate_fn(batch)에 뽑은 샘플을 전달
- collate_fn
  - text 추출 : PyTorch Tensor 형태로 리턴 받음
  - label 추출 : LongTensor를 통해 PyTorch Tensor로 추출한 label type 변경
- (texts, label) 튜플 반환
  - `train_loader`
  - `test_loader`

In [52]:
def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  # texts = torch.LongTensor(encode_bert(texts, max_len).input_ids)
  # labels = torch.LongTensor(labels)

  # encode_bert 함수에서 PyTorch Tensor type으로 바로 return했기에, LongTensor 처리가 불필요하다
  texts = encode_bert(texts, max_len, True).input_ids
  labels = torch.LongTensor(labels)

  return texts, labels

mini_batch=64

train_loader = DataLoader(train_ds, mini_batch, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, mini_batch, shuffle=False, collate_fn=collate_fn)

# for inputs, labels in train_loader:
#   print(inputs.shape)
#   print(labels.shape)
#   print(inputs)
#   print(labels)
#   break

# train_loader 내부를 한 배치만 보기
inputs, labels = next(iter(train_loader))
print(f'input size: {inputs.shape}\n')
print(f'label size: {labels.shape}\n')
print(f'input[0]: {inputs[0]}\n')
print(f'decode input[0]:\n {tokenizer.decode(inputs[0])}')
print(f"emotion of decode input[0]:\n {'부정' if labels[0].item() == 0 else '긍정'}")

input size: torch.Size([64, 400])

label size: torch.Size([64])

input[0]: tensor([  101,  2023,  2003,  2056,  2000,  2022,  1037,  3167,  2143,  2005,
         2848, 22132,  5280, 18891, 10649,  1012,  2002,  2241,  2009,  2006,
         2010,  2166,  2021,  2904,  2477,  2105,  2000,  4906,  1996,  3494,
         1010,  2040,  2024, 18145,  1012,  2122, 18145,  3058,  3376,  4275,
         1998,  2031,  2053,  3291,  2893,  2068,  1012,  4165,  2062,  2066,
         1037, 19965, 18286, 12127,  2084,  1037,  6317,  1010,  2987,  1005,
         1056,  2009,  1029,  2023,  2972,  3185,  2001,  2517,  2011,  2848,
         1010,  1998,  2009,  3065,  2129,  2041,  1997,  3543,  2007,  2613,
         2111,  2002,  2001,  1012,  2017,  1005,  2128,  4011,  2000,  4339,
         2054,  2017,  2113,  1010,  1998,  2002,  2106,  2008,  1010,  5262,
         1012,  1998,  3727,  1996,  4378, 11471,  1998,  5457,  1010,  1998,
         9981,  1010,  2005,  2008,  3043,  1012,  2023,  2003,  10


## Self-attention

이번에는 self-attention을 구현해보겠습니다.
Self-attention은 shape이 (B, S, D)인 embedding이 들어왔을 때 attention을 적용하여 새로운 representation을 만들어내는 module입니다.
여기서 B는 batch size, S는 sequence length, D는 embedding 차원입니다.
구현은 다음과 같습니다.

In [ ]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_model)

    if mask is not None:
      score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)
    result = self.dense(result)

    return result

대부분은 Transformer 챕터에서 배운 수식들을 그대로 구현한 것에 불과합니다.
차이점은 `mask`의 존재여부입니다.
이전 챕터에서 우리는 가변적인 text data들에 padding token을 붙여 하나의 matrix로 만든 방법을 배웠습니다.
실제 attention 계산에서는 이를 무시해주기 위해 mask를 만들어 제공해주게 됩니다.
여기서 mask의 shape은 (B, S, 1)로, 만약 `mask[i, j] = True`이면 그 변수는 padding token에 해당한다는 뜻입니다.
이러한 값들을 무시해주는 방법은 shape이 (B, S, S)인 `score`가 있을 때(수업에서 배운 $A$와 동일) `score[i, j]`에 아주 작은 값을 더해주면 됩니다. 아주 작은 값은 예를 들어 `-1000..00 = -1e9` 같은 것이 있습니다.
이렇게 작은 값을 더해주고 나면 softmax를 거쳤을 때 0에 가까워지기 때문에 weighted sum 과정에서 padding token에 해당하는 `v` 값들을 무시할 수 있게 됩니다.

다음은 self-attention과 feed-forward layer를 구현한 모습입니다.

In [ ]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    self.sa = SelfAttention(input_dim, d_model)
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

보시다시피 self-attention의 구현이 어렵지, Transformer layer 하나 구현하는 것은 수업 때 다룬 그림과 크게 구분되지 않는다는 점을 알 수 있습니다.

## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [ ]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


Positional encoding은 `angle_rads`를 구현하는 과정에서 모두 구현이 되었습니다. 여기서 `angle_rads`의 shape은 (S, D)입니다.
우리는 일반적으로 batch로 주어지는 shape이 (B, S, D)인 tensor를 다루기 때문에 마지막에 None을 활용하여 shape을 (1, S, D)로 바꿔주게됩니다.

위에서 구현한 `TransformerLayer`와 positional encoding을 모두 합친 모습은 다음과 같습니다:

In [ ]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, 1)

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0]
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 2, 32)

기존과 다른 점들은 다음과 같습니다:
1. `nn.ModuleList`를 사용하여 여러 layer의 구현을 쉽게 하였습니다.
2. Embedding, positional encoding, transformer layer를 거치고 난 후 마지막 label을 예측하기 위해 사용한 값은 `x[:, 0]`입니다. 기존의 RNN에서는 padding token을 제외한 마지막 token에 해당하는 representation을 사용한 것과 다릅니다. 이렇게 사용할 수 있는 이유는 attention 과정을 보시면 첫 번째 token에 대한 representation은 이후의 모든 token의 영향을 받습니다. 즉, 첫 번째 token 또한 전체 문장을 대변하는 의미를 가지고 있다고 할 수 있습니다. 그래서 일반적으로 Transformer를 text 분류에 사용할 때는 이와 같은 방식으로 구현됩니다.

## 학습

학습하는 코드는 기존 실습들과 동일하기 때문에 마지막 결과만 살펴보도록 하겠습니다.

In [ ]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [ ]:
n_epochs = 5

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 7.381510192528367
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   1 | Train Loss: 0.004828333854675293
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   2 | Train Loss: 1.4901161193847656e-08
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   3 | Train Loss: 1.4901161193847656e-08
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   4 | Train Loss: 1.4901161193847656e-08
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   5 | Train Loss: 1.4901161193847656e-08
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   6 | Train Loss: 1.4901161193847656e-08
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   7 | Train Loss: 2.8049244704675402e-08
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   8 | Train Loss: 1.4901161193847656e-08
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   9 | Train Loss: 1.4901161193847656e-08
=========> Train acc: 1.000 | Test acc: 1.000
Epoch  10 | Train Loss: 1.4901161193847656e-08


KeyboardInterrupt: 